### Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
# !pip install langchain==0.0.162

In [ ]:
# Run in a Colab shell cell (prefix with !)
!pip install -q langchain
!pip install tqdm

In [ ]:
!pip install rouge-score textstat


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3c977ba5c04173c54c8aa7dd4f1a05efbb5ac191a594c579bb9f968ab621b3e9
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
# Example: My preferred approach
import os
# Settings -> Developer Settings -> Personal Access Tokens -> Token (classic)
os.environ['GITHUB_TOKEN'] = "your_token_here"

GITHUB_USERNAME = "tienvu95"
REPO_NAME       = "reflexion_cp"
TOKEN = os.environ.get("GITHUB_TOKEN")
repo_url        = f"https://{TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
!git clone {repo_url}

Cloning into 'reflexion_cp'...
remote: Enumerating objects: 545, done.
remote: Counting objects: 100% (545/545), done.
remote: Compressing objects: 100% (236/236), done.
remote: Total 545 (delta 309), reused 540 (delta 304), pack-reused 0 (from 0)
Receiving objects: 100% (545/545), 8.09 MiB | 9.67 MiB/s, done.
Resolving deltas: 100% (309/309), done.


In [ ]:
!cd {REPO_NAME} && git pull

Already up to date.


Or in the submitted version, you can cd to the reflexion_cp folder, which is attached in the project repo


In [ ]:
cd reflexion_cp

/


In [ ]:
ls

bin@                        lib32@                    root/
boot/                       lib64@                    run/
content/                    libx32@                   sbin@
cuda-keyring_1.1-1_all.deb  media/                    srv/
datalab/                    mnt/                      sys/
dev/                        NGC-DL-CONTAINER-LICENSE  tmp/
etc/                        opt/                      tools/
home/                       proc/                     usr/
kaggle/                     python-apt/               var/
lib@                        python-apt.tar.xz*


# HF Transformer

In [ ]:
from huggingface_hub import login

# Login for Llama 3.1 access
login(token = "your_token_here")


Load LLama 3.1 8b Instruct


In [ ]:
# Single-block Colab snippet — paste & run
import os, traceback
import torch

# 1) config
MODEL_ID = os.environ.get("HF_MODEL_ID", "meta-llama/Meta-Llama-3.1-8B-Instruct")
HF_TOKEN = "your_token_here"
TRY_4BIT = True     # set False if bitsandbytes install / CUDA issues occur
MAX_NEW_TOKENS = 256

print("Device available:", "cuda" if torch.cuda.is_available() else "cpu")

# 2) install trust_remote_code usage requires hf token for gated Llama-3.1; if gated, set HF_TOKEN in env
if HF_TOKEN:
    os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN

# 3) load transformers model (quantized via bitsandbytes if requested)
from transformers import AutoTokenizer, AutoModelForCausalLM
bnb_cfg = None
if TRY_4BIT:
    try:
        from transformers import BitsAndBytesConfig
        bnb_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
        )
        print("BitsAndBytesConfig prepared for 4-bit.")
    except Exception as e:
        print("Could not prepare BitsAndBytesConfig (will run without 4-bit):", e)
        bnb_cfg = None
        TRY_4BIT = False

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("Loading model (this may take a few minutes)...")
load_kwargs = {"device_map": "auto", "trust_remote_code": True}
if TRY_4BIT and bnb_cfg is not None:
    load_kwargs["quantization_config"] = bnb_cfg
# use float16 for GPU
load_kwargs["torch_dtype"] = torch.float16

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, **load_kwargs)
model.eval()
print("Model loaded. Model device(s):", {k: v.device for k, v in model.named_parameters() if k in list(model.state_dict().keys())[:3] } )

# 4) Wrap with repo’s adapter (it accepts model+tokenizer)
from hotpotqa_runs.hf_transformers_llm import HFTransformersLLM
llm = HFTransformersLLM(model=model, tokenizer=tokenizer, device="cuda" if torch.cuda.is_available() else "cpu")
llm.max_new_tokens = MAX_NEW_TOKENS
llm.temperature = 0.0
print("LLM wrapper ready:", llm)

Device available: cuda
BitsAndBytesConfig prepared for 4-bit.
Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model loaded. Model device(s): {'model.embed_tokens.weight': device(type='cuda', index=0), 'model.layers.0.self_attn.q_proj.weight': device(type='cuda', index=0)}
LLM wrapper ready: <hotpotqa_runs.hf_transformers_llm.HFTransformersLLM object at 0x7a75cd4fcce0>


#Reflextion run

Some of the useful arguments:


*   Limit: number of examples you want to test
*   reflexion_strategy: 'reflexion' or 'none'
*   print_debug: set 'True' will let you see the entire thought process of the Reflexion



In [ ]:
from types import SimpleNamespace
from hotpotqa_runs import run_pubmedqa

args = SimpleNamespace(
    dataset='qiaojin/PubMedQA',
    dataset_config='pqa_labeled',
    split='train',
    limit=5,  # small test
    agent='cot',
    reflexion_strategy='reflexion',
    model=MODEL_ID,
    use_transformers=True,
    use_unsloth=False,
    hf_token=None,
    out=None,
    question_field='question',
    context_field='context',
    answer_field='final_decision',
    long_answer_field='long_answer',
    max_steps=6,
    print_debug=False,
    print_logit_debug=False,     # enable to see the yes/no/maybe probs (useful for Brier)
    keep_fewshot_examples=True,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    load_in_4bit=TRY_4BIT,
    max_seq_length=8192,
    max_reflect_attempts=3,
    force_finish_format=False,
    force_argmax_final=False,
    # NEW: readability / rewrite controls
    readability_min=6.0,        # Flesch-Kincaid lower bound (grade)
    readability_max=14.0,        # Flesch-Kincaid upper bound (grade)
    rewrite_on_readability=True, # if FK outside bounds, ask LLM to rewrite (keeps label fixed),
    enforce_readability_reflexion=True,  # trigger agent.reflect + rerun to enforce FK
    enforce_length=False,                 # try to match rationale length to long_answer
    # length_tolerance=0.20,               # +/-20% around gold long_answer word count
    max_readability_rewrites=1,          # Option A acceptance loop attempts
    rouge_drop_threshold=0.10,           # Accept rewrite only if ROUGE-1 drop <= this
    start_index = 500,
    end_index = 799
)

print("Calling run_pubmedqa.run(...) with external_llm (this will call the model)...")
try:
    results = run_pubmedqa.run(args, external_llm=llm)
    print("Runner finished. Sample result(s):")
    try:
        for r in (results[:5] if hasattr(results, "__len__") else [results]):
            print(r)
    except Exception:
        print(results)
except Exception as e:
    print("Runner error:", type(e).__name__, e)
    traceback.print_exc()
    # If you see 'apply_qkv' errors: try upgrading transformers or set TRY_4BIT=False
    print("\nIf you hit 'LlamaAttention.apply_qkv' or model-code errors, try:\n"
          "  pip install -U transformers accelerate safetensors\n"
          "and re-run with TRY_4BIT=False.\n")

Calling run_pubmedqa.run(...) with external_llm (this will call the model)...
Loading dataset qiaojin/PubMedQA split=train...


README.md: 0.00B [00:00, ?B/s]

pqa_labeled/train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Cropped dataset to indices [500:799] -> 300 examples.
Using fields -> question=question, context=context, answer=final_decision, long=long_answer
Using externally provided LLM instance (wrapped)


...

Enforcing top-logit label "no" with prob 0.8567
--> enforced_label set (attempt-level) = no


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Will save predicted_answer="no" (gold="maybe")
Final Answer: no
Rationale: The study found that MBSR reduced symptoms of depression and anxiety in patients with chronic pain. This suggests that MBSR may be an effective treatment for these symptoms.  #1553.

===== Example 300/300 =====
Question: Does telmisartan prevent hepatic fibrosis in rats with alloxan-induced diabetes?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Will save predicted_answer="yes" (gold="yes")
Progress: 300/300 (100.0%)  Acc=0.847
Final Answer: yes
Rationale: Observation: Answer is CORRECT
Done. Processed=300. Correct=254. Results saved to results_qiaojin_PubMedQA_train.csv
Accuracy: 0.8467
Macro-F1: 0.7004
              precision    recall  f1-score   support

       maybe     0.6667    0.2500    0.3636        24
          no     0.7717    0.9074    0.8340       108
         yes     0.9146    0.8929    0.9036       168

    accuracy                         0.8467       300
   macro avg     0.7843    0.6834    0.7004       300
weighted avg     0.8433    0.8467    0.8354       300

Mean Brier (0-2 scale over 300 examples): 0.304182
Avg ROUGE-1 F1 (rationale vs long_answer): 0.2147
Avg ROUGE-L F1 (rationale vs long_answer): 0.1505
Avg Flesch Reading Ease (rationales): 22.60
Avg Flesch-Kincaid Grade (rationales): 15.73
Avg SMOG Index (rationales): 14.78
Runner finished. Sample result(s):
None


# Baseline code

#Just predict yes no maybe

In [ ]:
# PubMedQA — single-token label + Brier from logits (Unsloth one-by-one)

import re, gc, torch
from datasets import load_dataset
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report

DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
MAX_INP = 768      # lower -> faster prefill
SYSTEM  = "Answer ONLY with one word: yes, no, or maybe."
CLASSES = ["yes","no","maybe"]
CLASS_SET = set(CLASSES)

# ---------- prompt: make the first generated token be the label ----------
def build_messages(q, ctx):
    if isinstance(ctx, list): ctx = " ".join(ctx)
    return [{
        "from": "human",
        "value": (
            f"{SYSTEM}\n\nQuestion: {q}\n\nAbstract:\n{ctx}\n\n"
            "Answer strictly with one word.\n\n"
            "Label: "   # <- next token will be yes|no|maybe
        )
    }]

def prompt_text(row):
    return tokenizer.apply_chat_template(
        build_messages(row["question"], row["context"]),
        tokenize=False, add_generation_prompt=True
    )

# ---------- tiny helpers ----------
def parse_label_token(tok_text: str) -> str:
    lab = tok_text.strip().strip(",.?:;!").lower()
    return lab if lab in CLASS_SET else "maybe"

def _first_token_ids(strings):
    out = []
    for s in strings:
        ids = tokenizer(s, add_special_tokens=False).input_ids
        if ids: out.append(ids[0])
    return out

# include space/no-space + case variants (Llama tokenizers often use leading-space tokens)
CAND_IDS = {
    "yes":   _first_token_ids([" yes","Yes","yes"]),
    "no":    _first_token_ids([" no","No","no"]),
    "maybe": _first_token_ids([" maybe","Maybe","maybe"]),
}

def probs_from_first_step_logits(out_struct):
    logits = out_struct.scores[0][0]    # (vocab,)
    pv = torch.softmax(logits, dim=-1)
    mass = {
        lab: float(pv[torch.tensor(ids, device=pv.device)].sum().item()) if ids else 0.0
        for lab, ids in CAND_IDS.items()
    }
    Z = sum(mass.values()) + 1e-12
    return {k: v/Z for k, v in mass.items()}

def brier_multiclass_sum(prob_dict, gold_label, classes=CLASSES):
    # Sum version ranges [0, 2] for 3 classes (0 is perfect)
    return sum((prob_dict[c] - (1.0 if c == gold_label else 0.0))**2 for c in classes)

# ---------- data ----------
ds   = load_dataset("qiaojin/PubMedQA", "pqa_labeled")
pmqa = ds["train"]       # use "test" for reporting; change to "train" if you want
N    = len(pmqa)        # set smaller for a smoke test

preds, golds = [], []
brier_probs, brier_vals = [], []

for i in tqdm(range(N), desc="Label-only + Brier (one-by-one)", ncols=100):
    row  = pmqa[i]
    gold = row["final_decision"].lower()
    golds.append(gold)

    prompt = prompt_text(row)
    enc = tokenizer(prompt, return_tensors="pt",
                    padding=False, truncation=True, max_length=MAX_INP).to(DEVICE)

    with torch.inference_mode():
        out = model.generate(
            **enc,
            max_new_tokens=1,            # exactly the label token
            do_sample=False, temperature=0.0,
            use_cache=False,             # lower KV mem
            pad_token_id=tokenizer.eos_token_id,
            output_scores=True,          # <- logits for first step
            return_dict_in_generate=True
        )

    # decode ONLY the new token
    new_tok = out.sequences[0, enc.input_ids.shape[1]:]
    label_text = tokenizer.decode(new_tok, skip_special_tokens=True)
    pred = parse_label_token(label_text)
    preds.append(pred)

    # probs -> Brier
    probs = probs_from_first_step_logits(out)
    brier_probs.append(probs)
    brier_vals.append(brier_multiclass_sum(probs, gold))

    del enc, out, new_tok
    if (i+1) % 100 == 0 and torch.cuda.is_available():
        torch.cuda.empty_cache(); gc.collect()

# ---------- metrics ----------
print(f"\nAccuracy:  {accuracy_score(golds, preds):.4f}")
print(f"Macro-F1:  {f1_score(golds, preds, average='macro'):.4f}\n")
print(classification_report(golds, preds, digits=4))

print(f"\nMean Brier (sum, 0–2): {sum(brier_vals)/len(brier_vals):.6f}")

# peek a few
for j in range(min(5, N)):
    print(f"[{j:03d}] gold={golds[j]:<6} pred={preds[j]:<6}  probs={brier_probs[j]}")


# predict + compute readability, brier score and  rogue1

In [ ]:
pip install rouge_score

In [ ]:
import re, gc, torch
from datasets import load_dataset
from tqdm.auto import tqdm
from statistics import mean

# Optional deps (install once in Colab if missing)
try:
    from evaluate import load as load_metric
except Exception:
    !pip -q install evaluate
    from evaluate import load as load_metric

try:
    import textstat
except Exception:
    !pip -q install textstat
    import textstat

DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
# ==== PubMedQA rationale-only eval (ROUGE-1 + readability) ====
# Assumes:
#   - model, tokenizer already loaded (Unsloth)
#   - tokenizer has chat template set (e.g., get_chat_template(..., "llama-3.1"))
#   - FastLanguageModel.for_inference(model) already called



# ---- knobs (no abstract shrinking) ----
MAX_INP = 1024    # cap for (prompt + abstract) tokens
MAX_NEW = 160     # generation budget for rationale

# ---- Single-pass prompt: ONLY rationale required ----
INSTR = (
    "You are answering PubMedQA. "
    "Write a concise explanation in plain language based only on the abstract. "
    "End with: 'This is not medical advice.'\n\n"
    "Return answers in this EXACT format:\n"
    "Reason:\n"
    "<your explanation>"
)

def build_messages(q, ctx):
    if isinstance(ctx, list): ctx = " ".join(ctx)
    return [{
        "from": "human",
        "value": f"{INSTR}\n\nQuestion: {q}\n\nAbstract:\n{ctx}\n\nReason:\n"
    }]

def apply_tpl(msgs):
    return tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)

# ---- helpers to clean and extract the rationale ----
ASSIST_RE = re.compile(
    r'^(?:<\|assistant\|>|<\|start_header_id\|>\s*assistant\s*<\|end_header_id\|>|assistant:?)[\s\r\n]*',
    re.IGNORECASE
)
def strip_assistant_header(text: str) -> str:
    text = text.lstrip()
    text = ASSIST_RE.sub("", text)
    lines = [ln.strip() for ln in text.splitlines()]
    while lines and re.fullmatch(r'(?:assistant:?|<\|assistant\|>)', lines[0], re.IGNORECASE):
        lines.pop(0)
    return "\n".join(lines).strip()

def parse_reason(out_text: str) -> str:
    t = strip_assistant_header(out_text)
    m = re.search(r'(?mi)^Reason:\s*(.*)$', t, flags=re.DOTALL)
    if m:
        return m.group(1).strip()
    # fallback: everything after the last "Reason:"
    cut = t.lower().rfind("reason:")
    return t[cut+len("reason:"):].strip() if cut != -1 else t

# ---- load data ----
ds   = load_dataset("qiaojin/PubMedQA", "pqa_labeled")
pmqa = ds["train"]      # change to "test" if you want test-set numbers
N    = len(pmqa)        # set smaller for a smoke test, e.g., N = 100

# ---- prebuild prompts (saves a little time) ----
prompts = [apply_tpl(build_messages(ex["question"], ex["context"])) for ex in pmqa]

# ---- generation loop ----
refs_long, hyps_long = [], []
for i in tqdm(range(N), desc="Generating rationales", ncols=100):
    ex = pmqa[i]
    refs_long.append((ex.get("long_answer") or "").strip())

    p = prompts[i]
    enc = tokenizer(
        p, return_tensors="pt",
        padding=False, truncation=True, max_length=MAX_INP
    ).to(DEVICE)

    with torch.inference_mode():
        out = model.generate(
            **enc,
            max_new_tokens=MAX_NEW,
            do_sample=False, temperature=0.0,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
        )

    # decode only new tokens
    new_tokens = out.sequences[0, enc.input_ids.shape[1]:]
    gen_text   = tokenizer.decode(new_tokens, skip_special_tokens=True)
    rationale  = parse_reason(gen_text)
    hyps_long.append(rationale)

    # free per-iteration
    del enc, out, new_tokens
    if (i+1) % 100 == 0 and torch.cuda.is_available():
        torch.cuda.empty_cache(); gc.collect()

# ---- evaluation: ROUGE-1 + readability ----
rouge = load_metric("rouge")
r = rouge.compute(predictions=hyps_long, references=refs_long, use_stemmer=True)
print(f"\nROUGE-1 (rationales vs gold long_answer), n={len(hyps_long)}: {float(r['rouge1']):.6f}")

fre  = [textstat.flesch_reading_ease(h) for h in hyps_long]
fk   = [textstat.flesch_kincaid_grade(h) for h in hyps_long]
smog = [textstat.smog_index(h) for h in hyps_long]
print("\nReadability of generated rationales (mean):")
print(f"  Flesch Reading Ease:   {mean(fre):.2f}")
print(f"  Flesch-Kincaid Grade:  {mean(fk):.2f}")
print(f"  SMOG Index:            {mean(smog):.2f}")

# ---- show first 3 model answers for sanity ----
for j in range(min(3, N)):
    ex  = pmqa[j]
    q   = ex["question"]
    ref = refs_long[j]
    hyp = hyps_long[j]
    print(f"\n[{j:03d}]")
    print("Q:", q)
    print("Gold (first 220ch):", (ref[:220] + "…") if len(ref) > 220 else ref)
    print("Hyp  (first 220ch):", (hyp[:220] + "…") if len(hyp) > 220 else hyp)


UndefinedError: 'dict object' has no attribute 'role'

In [ ]:
ref

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme)
</div>
